In [ ]:
import cv2
import numpy as np
import pandas as pd

# Funkcija za brojanje plavih objekata koji prelaze sredinu slike
def count_blue_objects_crossing_center(video_path, show_frames=True, roi=None, skip_frames=3):
    cap = cv2.VideoCapture(video_path)
    beetle_count = 0
    counted_ids = set()  # Set za objekte koji su već prebrojani
    skip_counter = 0

    # Background subtractor za detekciju pokreta
    fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

    # Uzimamo prvi frejm za dimenzije
    ret, first_frame = cap.read()
    if not ret:
        print("Ne mogu da učitam prvi frejm.")
        return 0

    # Ako ROI nije definisan, uzima se ceo frejm
    if roi is None:
        roi = (0, 0, first_frame.shape[1], first_frame.shape[0])  # (x, y, width, height)

    x_roi, y_roi, w_roi, h_roi = roi

    # Definisanje opsega plave boje u HSV
    lower_blue = np.array([60, 110, 150])  # Donja granica (svetlo plava) 60 110 150
    upper_blue = np.array([82, 160, 172])  # Gornja granica (tamno plava) 82 170 170

    center_x = w_roi // 2  # Sredina kropovanog frejma

    frame_number = 0  # Brojač frejmova
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_number += 1  # Ažuriraj broj trenutnog frejma

        # Proveri da li smo u fazi preskakanja frejmova
        if skip_counter > 0:
            skip_counter -= 1
            continue

        # Kropovanje trenutnog frejma prema ROI
        cropped_frame = frame[y_roi:y_roi+h_roi, x_roi:x_roi+w_roi]

        # Primena background subtractor-a
        fgmask = fgbg.apply(cropped_frame)
        fgmask = cv2.medianBlur(fgmask, 5)  # Uklanjanje šuma

        # Binarizacija za uklanjanje neželjenih pikselacija
        _, fgmask = cv2.threshold(fgmask, 25, 255, cv2.THRESH_BINARY)

        # Pronalazak kontura u maski pokreta
        contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        frame_has_contours = False  # Indikator da li trenutni frejm ima konture

        for cnt in contours:
            if 1000 < cv2.contourArea(cnt) < 4000:  # Normalna kontura
                x, y, w, h = cv2.boundingRect(cnt)
                obj_id = (x, y, w, h)

                # Računanje centra objekta
                center_of_object = x + w // 2 - 120
                cv2.line(cropped_frame, (center_of_object, 0), (center_of_object, h_roi), (255, 0, 0), 2)
                # Proveri da li centar objekta prelazi sredinu i da li nije već prebrojan
                if center_x - 6 <= center_of_object <= center_x + 6 and obj_id not in counted_ids:
                    # Ekstrakcija ROI i konverzija u HSV
                    roi_hsv = cv2.cvtColor(cropped_frame[y:y+h, x:x+w], cv2.COLOR_BGR2HSV)
                    mean_hsv = cv2.mean(roi_hsv)[:3]

                    # Provera da li je objekat u opsegu plave boje
                    if lower_blue[0] <= mean_hsv[0] <= upper_blue[0] and \
                       lower_blue[1] <= mean_hsv[1] <= upper_blue[1] and \
                       lower_blue[2] <= mean_hsv[2] <= upper_blue[2]:
                        counted_ids.add(obj_id)  # Dodaj objekat u set prebrojanih
                        beetle_count += 1

                        # Ispis HSV vrednosti
                        print(f"Frejm {frame_number}: Detektovan plavi objekat na ({x}, {y}, {w}, {h}), HSV: {mean_hsv}")

                        # Obeležavanje konture na kropovanom frejmu
                        cv2.rectangle(cropped_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Plavi pravougaonik
                        cv2.putText(cropped_frame, "Counted", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                        frame_has_contours = True

                        skip_counter = skip_frames # preskakanje broja frejmova

            elif 9000 < cv2.contourArea(cnt)< 12000 or 13000 < cv2.contourArea(cnt) < 40000:  # Velika kontura, podela na manje konture
                x, y, w, h = cv2.boundingRect(cnt)
                step_x = max(w // 3, 1)  # Razbijanje na 3 dela po širini
                step_y = max(h // 3, 1)  # Razbijanje na 3 dela po visini

                for i in range(0, w, step_x):
                    for j in range(0, h, step_y):
                        sub_x = x + i
                        sub_y = y + j
                        sub_w = min(step_x, w - i)  # Osiguravamo da ostanemo u okviru
                        sub_h = min(step_y, h - j)

                        sub_obj_id = (sub_x, sub_y, sub_w, sub_h)

                        # Računanje centra pod-regiona
                        center_of_object = sub_x + sub_w // 2 + 110
                        cv2.line(cropped_frame, (center_of_object, 0), (center_of_object, h_roi), (255, 0, 0), 2)
                        if center_x - 15 <= center_of_object <= center_x + 15 and sub_obj_id not in counted_ids:
                            roi_hsv = cv2.cvtColor(cropped_frame[sub_y:sub_y+sub_h, sub_x:sub_x+sub_w], cv2.COLOR_BGR2HSV)
                            mean_hsv = cv2.mean(roi_hsv)[:3]

                            if 56 <= mean_hsv[0] <= 82 and \
                               (110 <= mean_hsv[1] <= 130 or 140 <= mean_hsv[1] <= 160) and \
                               150 <= mean_hsv[2] <= 170:
                                counted_ids.add(sub_obj_id)  # Dodaj pod-region u set prebrojanih
                                beetle_count += 1

                                # Ispis HSV vrednosti
                                print(f"Frejm {frame_number}: Detektovan plavi objekat u pod-regiji ({sub_x}, {sub_y}, {sub_w}, {sub_h}), HSV: {mean_hsv}")

                                # Obeležavanje pod-regiona na kropovanom frejmu
                                cv2.rectangle(cropped_frame, (sub_x, sub_y), (sub_x + sub_w, sub_y + sub_h), (255, 0, 0), 2)
                                cv2.putText(cropped_frame, "Sub Counted", (sub_x, sub_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                                frame_has_contours = True

                                skip_counter = 15
                                

        # Prikaz samo frejmova koji imaju konture
        if frame_has_contours and show_frames:
            cv2.line(cropped_frame, (center_x, 0), (center_x, h_roi), (0, 255, 0), 2)  # Linija sredine
            cv2.imshow('Detected Blue Objects Crossing Center', cropped_frame)
            cv2.waitKey(800)  # Prikazuj frejm 500ms pre nego što pređe na sledeći

    cap.release()
    cv2.destroyAllWindows()
    return beetle_count

# Funkcija za izračunavanje MAE
def calculate_mae(predictions, ground_truth):
    errors = [abs(predictions[video] - ground_truth[video]) for video in ground_truth]
    mae = sum(errors) / len(errors)
    return mae

# Funkcija za procesiranje više videa i izračunavanje MAE
def process_videos_with_mae():
    beetle_counts = {}
    ground_truth = pd.read_csv("data/buzzy_beetle_count.csv")
    ground_truth_dict = dict(zip(ground_truth['video'], ground_truth['count']))

    for i in range(1, 11):  # Od video_1.mp4 do video_10.mp4
        video_path = f"data/video_{i}.mp4"
        print(f"\nProcessing {video_path}...")
        roi = (400, 250, 500, 320)  # Definisani region interesa (x, y, širina, visina)
        beetle_count = count_blue_objects_crossing_center(video_path, show_frames=True, roi=roi)
        beetle_counts[video_path.split('/')[-1]] = beetle_count
        print(f"Video {video_path}: {beetle_count} detektovanih plavih objekata.")

    print("\nRezultati za sve video fajlove:")
    for video, count in beetle_counts.items():
        print(f"{video}: {count} plavih objekata detektovano.")

    # Izračunavanje MAE
    mae = calculate_mae(beetle_counts, ground_truth_dict)
    print(f"\nMean Absolute Error (MAE): {mae}")

# Procesiraj sve video fajlove i izračunaj MAE
process_videos_with_mae()




Processing data/video_1.mp4...
Frejm 214: Detektovan plavi objekat na (344, 242, 56, 60), HSV: (78.92202380952382, 157.6669642857143, 164.9104166666667)
Frejm 218: Detektovan plavi objekat na (342, 244, 58, 58), HSV: (78.91319857312722, 157.72919143876337, 163.96135552913196)
Video data/video_1.mp4: 2 detektovanih plavih objekata.

Processing data/video_2.mp4...
Frejm 240: Detektovan plavi objekat u pod-regiji (132, 95, 41, 95), HSV: (57.61335044929397, 112.3198973042362, 163.18382541720155)
Video data/video_2.mp4: 1 detektovanih plavih objekata.

Processing data/video_3.mp4...
Frejm 145: Detektovan plavi objekat na (348, 236, 54, 50), HSV: (78.67888888888889, 154.41814814814813, 166.28555555555556)
Frejm 189: Detektovan plavi objekat na (344, 246, 56, 56), HSV: (79.828125, 155.2847576530612, 163.500318877551)
Video data/video_3.mp4: 2 detektovanih plavih objekata.

Processing data/video_4.mp4...
Frejm 168: Detektovan plavi objekat u pod-regiji (127, 100, 53, 100), HSV: (58.3649056603